## Design and implementation of an `Ansatz` class

The state of a quantum system is represented as a circuit on a quantum computer.
For a system with $N$ qubits, the circuit will consist of a certain number of
_gates_ arranged in $M$ _layers_.

<img src="https://user-images.githubusercontent.com/3708689/255829669-b989d9a7-714f-49d2-b59f-03829568ec30.png" width=300 height=300/>

The sample circuit in the figure shows:

* The qubits, circled in blue.
* A gate, circled in magenta.
* A layer, circled in green.

Quantum software development kits (SDKs), such as Qiskit, already provide
classes to describe the quantum circuit.
However, we need a further abstraction layer to represent the Ansatz for the
system.
Having a flexible abstract representation of the state is beneficial for, among
others, these reasons:

* Optimized circuits are difficult to interpret, so having both concrete (as circuit) and abstract (as Ansatz) representations can be beneficial for algorithmic analysis, physical and chemical insight.
* Transpiling from the state representation to the circuit implemented on hardware can be aggressively optimized. Both the efficiency of transpilation and the efficiency of the resulting circuits can make or break a variational quantum algorithm.

After discussions with the team, there's agreement that you should design, implement, and unit-test an `Ansatz` class that:

* Respects these constraints:

    1. Each layer can have multiple gates.
    2. No gates can overlap in a layer.
    3. Only 1- and 2-qubit gates are allowed. That is, in the figure above any gate can "touch" at most two qubits.

* Achieves these functional requirements:

    1. Fast access to the first available layer.
    2. Fast iteration qubit-by-qubit.
    3. Fast iteration layer-by-layer.

A gate is a wrapper type for a unitary matrix of dimension $2^q \times 2^q$, where $q$ is the number of active qubits for the gate, either 1 or 2, given the constraint above.

You can assume it's implemented as:

In [1]:
from dataclasses import dataclass

import numpy as np


@dataclass
class Gate:
    qubits: list[int]
    """List of active qubits for the gate."""
    unitary: np.ndarray
    """Gate unitary."""

1. What is your strategy to hash out the design for this class?
2. How would you actually implement your design? Please show relevant code snippets.
3. How would you test that your implementations meets the constraints and functional requirements? Please show relevant code snippets.
4. Explain how you would gather further feedback from the team and ask for clarifications on the specifications.

## <span style='color:blue'> Answer </span>

**1. What is your strategy to hash out the design for this class?**

Well, Ansatz referes to the quantum machine learning process and mainly it is about the variational quantum circuits. To deal with this problem I would like to start with the main three blocks that we have in a quantum device. First block is **encoding block** which we encod classical data to the quantum states. The second block is **Quantum model** which can either be *deterministic* or *variationl*. In this case we are dealing with **variational quantum model**. And the last block is our measurment block that the output tells us how much we are close to the true anwere.

As a simple example I would say that we can start with a bunch of qubits in random states and then apply various type of trainable-gates over them and following that we have measurment like the expection value of an operator. Then by comparison the measurment result by the true value and calculation the cost function we understand how we should train the gates in order to reach the zero value for cost function.

In this problem we just need to design an **Ansatz class** with specific constraints and requirements. 
* we need a class to represnt the states of qunatum system which involve multiple layers, where each layer contain a list of gates.
* we have a constrain about layer which each layer can contain multiple gates, but there should be no overlap between gates in a layer. Only 1- and 2-qubit gates are allowed
* we shoud achieves functional requirements like: fast access to the first available layer, fast iteration qubit-by-qubit, and fast iteration layer-by-layer.

**2. How would you actually implement your design? Please show relevant code snippets.**

To start, lets deal with this problem just by considering the constrain that we have:
1. Each layer can have multiple gates.
2. No gates can overlap in a layer.
3. Only 1- and 2-qubit gates are allowed. That is, in the figure above any gate can "touch" at most two qubits.

Intersting point here is about this constrains that all about the layers. Then in this step we have a specific outline for making the layers class.

In [2]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Gate:
    qubits: list[int]
    """List of active qubits for the gate (only one or two qubits)."""
    unitary: np.ndarray
    """Gate unitary."""

class Layer:

    def __init__(self):
        self.gates = [] 

    def add_gate(self, gate):
        if len(gate.qubits) not in [1,2]:
            raise ValueError('doesnt opperate over one or two active qubit')
        for exist_gate in self.gates:
            if set(gate.qubits) & set(exist_gate.qubits):
                raise ValueError('has overlap with others')
        self.gates.append(gate)
        return 'gate add successfully'


Now its the time to define **Ansatz class**. For making this part we should consider that at the end we must meet the functional requirements:

**Fast access to the first available layer.**

In this part, I would say that accessing to the first element of a list takes constant time in python. In this case regardless to the number of layers as we want to access to the first elemnt of list, we meet the target.

**Fast iteration layer-by-layer.**

A Practical way to iterate layer-by-layer is using the *yield* as instead of returning entire of a list at once, yield one at time. This is thefaset sturcure of  the code that we do iteration layer by layer and in each time we consider specific layer.

**Fast iteration qubit-by-qubit.**

Again using *yield* we will have iteration over qubits in an efficient way

In [3]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Gate:
    qubits: list[int]
    """List of active qubits for the gate (only one or two qubits)."""
    unitary: np.ndarray
    """Gate unitary."""

class Ansatz_Layer:
    '''In this version of layers we will ignore the wrong gates.'''
    def __init__(self):
        self.gates = [] 

    def add_gate(self, gate):
        if len(gate.qubits) not in [1,2]:
            return  
        for exist_gate in self.gates:
            if set(gate.qubits) & set(exist_gate.qubits):
                return
        self.gates.append(gate)
    def __iter__(self):
        '''For fast iteration over layers and qubits'''
        return iter(self.gates)
    
class Ansatz:

    def __init__(self):
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def get_layer(self):
        return self.layers
    
    def get_first_layer(self):
        if self.layers:
            return self.layers[0]
        else:
            return None
        
    def get_layer_iteration(self):
        '''return iter(self.layers)'''
        for layer in self.layers:
            yield layer
        
    def get_qubit_iteration(self):
        for layer in self.layers:
            for gate in layer:
                for qubit in gate.qubits:
                    yield qubit



**3. How would you test that your implementations meets the constraints and functional requirements? Please show relevant code snippets.** 

For test these classes I would like to do test over each classes in order to debuging. Let's begin by the **Layer class**.

In [4]:
layer = Layer()
def Layer_test(lst_gates):
    for i, gate in enumerate(lst_gates):
        try:
            layer.add_gate(gate)
        except ValueError as e:
            print(f'there is a problem: gate {i} {e}')

by defining a couple of gates we try to test the layer class

In [5]:
gate0 = Gate([0], np.identity(2))
gate1 = Gate([0,1,3], np.identity(8))
gate2 = Gate([1,2], np.identity(4))
gate3 = Gate([0,2], np.identity(4))

gates = [gate0, gate1, gate2, gate3]

Layer_test(gates)

there is a problem: gate 1 doesnt opperate over one or two active qubit
there is a problem: gate 3 has overlap with others


Here we can see that gate 1 and gate 3 have problems and we expect that they don-t append to the layer. Let's have a look

In [6]:
layer.gates

[Gate(qubits=[0], unitary=array([[1., 0.],
        [0., 1.]])),
 Gate(qubits=[1, 2], unitary=array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]))]

Yeah, It is working.

Now let's do some test over the Ansatz class. Here we should bear in mind that the layer class is improved by some corrections.

In [7]:
gate0 = Gate([0], np.identity(2))
gate1 = Gate([0,1,2], np.identity(8))
gate2 = Gate([1,2], np.identity(4))
gate3 = Gate([0,2], np.identity(4))

layer1 = Ansatz_Layer()
layer2 = Ansatz_Layer()
layer3 = Ansatz_Layer()

layer1.add_gate(gate0)
layer1.add_gate(gate2)
layer2.add_gate(gate3) 
layer3.add_gate(gate1)

ansatz = Ansatz()
ansatz.add_layer(layer1) 
ansatz.add_layer(layer2)
ansatz.add_layer(layer3)

first_layer = ansatz.get_first_layer()

first_layer.gates

[Gate(qubits=[0], unitary=array([[1., 0.],
        [0., 1.]])),
 Gate(qubits=[1, 2], unitary=array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]))]

Lets test the code when we have wrong gate

In [8]:
layers = ansatz.get_layer()
layers[2].gates

[]

yep. we have an empty list

Lets have a look on get_layer_iteration function that is iteration layar-by-layer

In [9]:
for i, layer in enumerate(ansatz.get_layer_iteration()):
    print(f"Layer {i+1}:")
    for gate in layer:
        print(f"  Gate acting on qubits {gate.qubits}")


Layer 1:
  Gate acting on qubits [0]
  Gate acting on qubits [1, 2]
Layer 2:
  Gate acting on qubits [0, 2]
Layer 3:


Lets have a look on get_qubit_iteration function that is iteration qubit-by-qubit

In [10]:
for qubit in ansatz.get_qubit_iteration():
    print(qubit)

0
1
2
0
2


Which is completly True! in first layer we start by qubit 0,1,2 and then in the second layer we have qubit 0 and 2.